# Imports

In [1]:
import os
import glob
import tensorflow as tf
import pandas as pd
import numpy as np
import zipfile as zf
from glob import glob
# from keras.saving.hdf5_format import save_attributes_to_hdf5_group
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration, AutoTokenizer, TFBertModel, AutoModel, BertTokenizer, TFBertForQuestionAnswering
import sentencepiece
from metapub import PubMedFetcher
from semanticscholar import SemanticScholar
from metapub import FindIt
import requests
import urllib
import json
from transformers import BioGptModel, BioGptConfig, BioGptTokenizer, BioGptForCausalLM

2023-07-12 05:38:39.107584: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-12 05:38:39.230553: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-12 05:38:39.231876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 05:38:40.681594: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-12 05:38:46 ip-172-31-25-176 metapub.config[30523] WARNING NCBI_API_KEY was not set.


In [ ]:
attributes_csv = starter_data + "/final.csv"
attributes_df = pd.read_csv(attributes_csv)
attributes_df.head(10)

,Unnamed: 0,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,0,52273203,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/39063bcd...,Large conformational changes of a highly dynam...,"{""{\""authorId\"": \""51285510\"", \""name\"": \""Isa...",Communications Biology,069e05e7-ca35-41d0-a8c7-bdc9ec6a82af,2018.0,28,13,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2018-09-03,"{""name"": ""Communications Biology"", ""pages"": nu...",2023-03-31T01:28:37.897Z,5692596
1,1,23708908,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/6ec7c156...,Drug sensitivity profiling identifies potentia...,"{""{\""authorId\"": \""5699001\"", \""name\"": \""H. K...",OncoTarget,106ecc86-1801-4b36-a05c-d36db69a968c,2017.0,46,24,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2017-10-31,"{""name"": ""Oncotarget"", ""pages"": ""97516 - 97527...",2023-04-05T06:18:15.837Z,5692609
2,2,13232625,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/c8566272...,Men's memory for women's sexual-interest and r...,"{""{\""authorId\"": \""4425040\"", \""name\"": \""T. T...",NaN,NaN,2011.0,36,13,0,True,"{""{\""category\"": \""Psychology\"", \""source\"": \...",{Review},2011-09-01,"{""name"": ""Applied Cognitive Psychology"", ""page...",2023-04-05T08:18:02.240Z,5692611
3,3,73484844,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/d4c9b2fa...,The Reactive Plasticity of Hippocampal Ionotro...,"{""{\""authorId\"": \""2225106\"", \""name\"": \""A. M...",International Journal of Molecular Sciences,8506a01a-40b8-4e6f-bbb8-ce2492139c15,2019.0,97,15,1,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...","{Review,JournalArticle}",2019-02-27,"{""name"": ""International Journal of Molecular S...",2023-05-20T17:20:54.230Z,5692615
4,4,229159752,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/7f13abe2...,High Mortality Rate in Adult COVID-19 Inpatien...,"{""{\""authorId\"": \""7685173\"", \""name\"": \""Saee...",Journal of Multidisciplinary Healthcare,12e5e550-28bd-401c-a348-f347956deec4,2020.0,37,17,1,True,"{""{\""category\"": \""Medicine\"", \""source\"": \""s...","{JournalArticle,Review}",2020-12-01,"{""name"": ""Journal of Multidisciplinary Healthc...",2023-05-23T13:34:07.581Z,5692620
5,5,10041098,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/92b26dee...,NADH-dependent metabolism of nitric oxide in a...,"{""{\""authorId\"": \""1792255\"", \""name\"": \""A. I...",Planta,66507f0b-12a1-4058-a35c-73029853d5b9,2004.0,30,145,3,False,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2004-01-22,"{""name"": ""Planta"", ""pages"": ""95-102"", ""volume""...",2023-04-05T15:01:41.273Z,5692625
6,6,19472375,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/2c682757...,Aspects of posttraumatic stress disorder in lo...,"{""{\""authorId\"": \""2000616\"", \""name\"": \""A. D...",Journal of cancer survivorship,649b7693-cb10-4892-b8a2-65b9b51fcd50,2016.0,33,18,0,False,"{""{\""category\"": \""Psychology\"", \""source\"": \...",{JournalArticle},2016-02-26,"{""name"": ""Journal of Cancer Survivorship"", ""pa...",2023-04-05T19:07:12.948Z,5692626
7,7,219603447,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/9b9659a4...,Multicenter assessment of the Brain Injury Gui...,"{""{\""authorId\"": \""145776250\"", \""name\"": \""Ab...",Trauma Surgery & Acute Care Open,d40f33e8-6f79-4e91-a62e-acf29f9a3e2f,2020.0,35,11,0,True,"{""{\""category\"": \""Medicine\"", \""source\""

## Load Models & Tokenizers

In [ ]:
max_sequence_length = 512
embedding_size = 200

In [ ]:
bio_bert_model = AutoModel.from_pretrained("gsarti/biobert-nli")
bio_bert_tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
original_bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
T5Abstract_model = TFT5ForConditionalGeneration.from_pretrained('t5-base')
T5tokens = T5Tokenizer.from_pretrained('t5-base')

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
### T5 Abstractive Text Summarization Model
def t5summary_model(tokenizer, text, t5model):
    summarize = "summarize: "
    encoding = tokenizer([summarize+text], return_tensors='tf')
    output = t5model.generate(encoding.input_ids,
                             num_beams=3,
                            no_repeat_ngram_size=2,
                              top_k=10,
                              top_p=80,
                              max_length=50,
                              min_length=30
                             )
    print([tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in output])
#     print(summary)

In [ ]:
t5summary_model(T5tokens, text_example, T5Abstract_model)

['a clear and simple standard for software engineering and management requirements remains elusive. the paper will make an admittedly bold and brash attempt to boil it all down into something anyone can understand, hopefully resulting in an brief reference']


### Q&A Model

In [ ]:
bertqa = TFBertForQuestionAnswering.from_pretrained('bert-base-uncased')
berttokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sample_question = "question: What is the capital city of Switzerland?"
sample_context = "context: Many people are surprised when they hear that pretty little Bern is the Swiss capital. Surely industrial Zurich or international Geneva would be more logical, they say. But it's precisely to avoid a concentration of power that Bern was chosen as the “federal city” exactly 170 years ago."

tokenized_qa = berttokenizer([sample_question + sample_context], return_tensors ='tf')
model_output = bertqa(**tokenized_qa)

answer_start_index = int(tf.math.argmax(model_output.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(model_output.end_logits, axis=-1)[0])
print(answer_end_index)

answer_encoded = tokenized_qa.input_ids[0,answer_start_index:answer_end_index]
start_answer = tokenized_qa.input_ids[0][answer_start_index]
print(berttokenizer.decode(answer_encoded, skip_special_tokens=True, clean_up_tokenization_spaces=True))

53
. but it's precisely to avoid a concentration of


In [ ]:
qa_tokens = T5tokens([sample_question + sample_context], return_tensors='tf')

t5_model_out = T5Abstract_model.generate(qa_tokens.input_ids,
                                         num_beams=3,
                                         no_repeat_ngram_size=2,
                                         max_length = 3,
                                         min_length=2)

print([T5tokens.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=True) for i in t5_model_out])

['Bern']


In [ ]:
from transformers import RobertaForQuestionAnswering, RobertaTokenizer, AutoTokenizer, TFAutoModel, RobertaForSequenceClassification

roberta_tokens = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
robertaqa_model = TFAutoModel.from_pretrained("deepset/roberta-base-squad2")

roberta_tokenized_qa = roberta_tokens([sample_question + sample_context], padding=True,  return_tensors='tf')
roberta_model_layer = robertaqa_model(**roberta_tokenized_qa)


AttributeError: ignored

In [ ]:
tokenized_qa.input_ids[0]

<tf.Tensor: shape=(70,), dtype=int32, numpy=
array([  101,  3160,  1024,  2054,  2003,  1996,  3007,  2103,  1997,
        5288,  1029,  6123,  1024,  2116,  2111,  2024,  4527,  2043,
        2027,  2963,  2008,  3492,  2210, 16595,  2003,  1996,  5364,
        3007,  1012,  7543,  3919, 10204,  2030,  2248,  9810,  2052,
        2022,  2062, 11177,  1010,  2027,  2360,  1012,  2021,  2009,
        1005,  1055, 10785,  2000,  4468,  1037,  6693,  1997,  2373,
        2008, 16595,  2001,  4217,  2004,  1996,  1523,  2976,  2103,
        1524,  3599, 10894,  2086,  3283,  1012,   102], dtype=int32)>

In [ ]:
model_output

TFQuestionAnsweringModelOutput(loss=None, start_logits=<tf.Tensor: shape=(1, 67), dtype=float32, numpy=
array([[ 0.16508265,  0.45413932,  0.04580224,  0.09424645, -0.07182357,
        -0.11457947,  0.11367559,  0.12410395, -0.00738204,  0.29390356,
        -0.19900739, -0.12262025, -0.00352622, -0.26001504,  0.16652006,
         0.11015014, -0.05808127, -0.07685345, -0.05458946,  0.0009411 ,
        -0.01198626, -0.01611032, -0.11756027, -0.08084115,  0.04347808,
        -0.48372173, -0.09519623, -0.24002753,  0.12300973,  0.3441984 ,
         0.14760727,  0.06125803, -0.20339994,  0.13757387,  0.00848398,
        -0.21722472,  0.04151647, -0.19865523, -0.18272205, -0.48353076,
         0.01219941,  0.0961633 , -0.25447947, -0.24058512,  0.04575343,
        -0.48203897, -0.21114022, -0.5456536 , -0.5361859 , -0.62731385,
        -0.24357261, -0.1540298 ,  0.04120081,  0.18258283, -0.15227222,
        -0.01945501,  0.08738828, -0.1486671 ,  0.01810225, -0.10272819,
        -0.07920434,

In [ ]:
### BioBERT or BERT Q&A or Clincal-T5-Large
sample_question = "question: What is the capital city of Switzerland?"
sample_context = "context: Many people are surprised when they hear that pretty little Bern is the Swiss capital. Surely industrial Zurich or international Geneva would be more logical, they say. But it's precisely to avoid a concentration of power that Bern was chosen as the “federal city” exactly 170 years ago."

encoding_gpt = biogpttokenizer([sample_question + sample_context], return_tensors='tf')
encoding_gpt

# encoded_answer = biogptmodel.generate(encoding_gpt.input_ids,
                                      # num_beams = 3,
                                      # top_p = 80,
                                      # top_k = 4,
                                      # no_repeat_ngram_size = 2,
                                      # max_length = 10,
                                      # min_length =2
                                      # )
# decoded_answer = biogpttokenizer.decode(encoded_answer)
# print(decoded_answer)

{'input_ids': <tf.Tensor: shape=(1, 79), dtype=int32, numpy=
array([[    2,  2969,    20,  5732,    21,     6, 11991,  5712,     5,
        14693,   927,  1544,    20,  2621,   962,    31, 32991,  4464,
          154,   298,  3734,  1770,    22,   968,   765,  7484,  1161,
        14421,   169,    21,     6, 11029, 11991,     4,   250, 37144,
         5132,  1196,  1595, 10956,    30,  3560,  7661,  3838,    14,
          739,    33,    77, 10775,     7,   298, 20933,     4,  6349,
          114,   121,  8641,    13,  3350,    14,   276,     5,  1382,
           22, 14421,   169,    17,  4852,    27,     6,     3, 14191,
         5712,     3, 15362,  8093,   113,  5306,     4]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 79), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1,

### Extractive Summary Models

In [ ]:
### T5 or T5v1 or Clincal-T5-Large or Bio-GPT

### Classification Model

In [ ]:
### BERT or Bio-GPT

### NER Model

In [ ]:
### BioELECTRA